# <font color="#76b900">**Notebook 1:** Understanding Batching Strategies</font>

Welcome back. In this notebook you will dive deeper into the metrics, characterizing the speed of the LLM inference engine. You will explore the cutting-edge optimizations employed in modern inference engines, simulate their effects, and analyze the impact on the key metrics that matter most.

## Learning Objectives
By the end of this notebook, you will be able to:
- Understand and measure time to first token (TTFT), end-to-end latency (E2E Latency), and inter-token latency (ITL).
- Analyze throughput metrics and simulate their dependencies on various factors.
- Explore the impact of batching and inflight batching on GPU utilization and performance.
- Investigate the effects of concurrency settings on latency and throughput.

**Before starting this notebook, please make sure to watch its corresponding video.**

## Table of Contents

- [**Latency Metrics**](#Latency-Metrics)
  - [[EXCERCISE] Estimating Inter-Token Latency (ITL) For Human Perception](#[EXERCISE]-Estimating-Inter-Token-Latency-(ITL)-For-Human-Perception)
- [**Introducing the Simulator**](#Introducing-the-Simulator)
- [**Batching: The Key to Efficient Throughput**](#Batching:-The-Key-to-Efficient-Throughput)
  - [Memory-Bound and Compute-Bound Functions](#Memory-Bound-and-Compute-Bound-Functions)
  - [**Prefill:** A Compute-Bound Operation](#Prefill:-A-Compute-Bound-Operation)
  - [**Decoding:** A Memory-Bound Operation](#Decoding:-A-Memory-Bound-Operation)
  - [Arithmetic Intensity](#Arithmetic-Intensity)
  - [Simulating Infinitely Compute-Bound Prefill and Infinitely Memory-Bound Decoding](#Simulating-Infinitely-Compute-Bound-Prefill-and-Infinitely-Memory-Bound-Decoding)
  - [Adding Variable Output Length](#Adding-Variable-Output-Length)
- [**Throughput Metrics**](#Throughput-Metrics)
  - [Choosing The Right Throughput Metric](#Choosing-The-Right-Throughput-Metric)
  - [[EXCERCISE] Implement Metrics Printing and Throughput Measurement In the Simulation](#[EXERCISE]-Measuring-Throughput-and-Printing-Metrics)
- [**Inflight Batching (IFB)**](#Inflight-Batching-(IFB))
  - [IFB in simulation](#IFB-In-Simulation)
  - [Chunked Context](#Chunked-Context)
  - [[EXCERCISE] Batcher With Only One Prefill Per Batch](#[EXCERCISE]-Batcher-With-Only-One-Prefill-Per-Batch)
  - [Client-Side Concurrency](#Client-Side-Concurrency)
  - [Max Batch Size](#Max-Batch-Size)
- [**Additional Notes**](#Additional-Notes)
  - [On concurrency and request rate as a result metric](#On-concurrency-and-request-rate-as-a-result-metric)
  - [On concurrency and request rate as an input parameter](#On-concurrency-and-request-rate-as-an-input-parameter)
  - [[OPTIONAL EXCERCISE] Queue Growth](#[OPTIONAL-EXCERCISE]-Queue-Growth)


<br><hr>

## **Latency Metrics**
Several important metrics are available in the benchmarks:

* **TTFT (Time To First Token)**: measures the time it takes for the model to generate the first token of a response. You've experienced it in the previous notebook.
* **E2E Latency (End-to-End Latency)**: measures the total time it takes for the model to generate a complete response.
* **ITL (Inter-Token Latency)**: also known as Time Per Output Token (TPOT), measures the average time the client waits between consecutive tokens in a response in the streaming scenario.

To separate the prefill characteristics from the decoding ones, TTFT and ITL are reported independently in GenAI-Perf, as shown in the image below.

<img src="images/metrics.png" alt="metrics" width=1200px/>

### **[EXERCISE]** Estimating Inter-Token Latency (ITL) For Human Perception

Let us try to estimate typical inter-token latencies for human perception. Consider the following details:

- Normal reading for comprehension speed is about 200–230 words per minute (wpm). 
- Skimming speed is 700 wpm ([see wiki](https://en.wikipedia.org/wiki/Speed_reading#Skimming_and_scanning)). 
- We can assume that an arbitrary token accounts for around 0.75 words, on average (as a standard simplifying assumption used a lot for English).

Let's convert these to ITL-compatible units of ms/token. Using reasonable average input statistics, we can expect to get decent average output statistics via some simple unit conversion. 

In [1]:
def calculate_itl(wpm, words_per_token=0.75):
    ## TODO: Implement the method
    
    ## NOTE: 60 seconds to a minute, 1000 milliseconds to a second

    ## NOTE: Unit arithmetic of [t/s] = [(words/min) * (tokens/word) * (min/s)] = [(words/min) / (words/tokens) / (s/min)]
    tokens_per_second = 0

    ## NOTE: Unit arithmetic of [ms/t] = [(ms/s * s/token)] = [(ms/s) / (token/s)]
    inter_token_latency_ms = 0

    return inter_token_latency_ms

print(f"READING:  230 words per minute correspond to {calculate_itl(230):.0f}ms between tokens, on average")
print(f"SKIMMING: 700 words per minute correspond to {calculate_itl(700):.0f}ms between tokens, on average")

READING:  230 words per minute correspond to 0ms between tokens, on average
SKIMMING: 700 words per minute correspond to 0ms between tokens, on average


In the vast majority of the LLM inference setups, ITL is much lower than these reference values, typically around 20–40 ms. Still, it's important to keep these numbers in mind as minimum thresholds for comfort.

<details>
<summary><b>Reveal Solution</b></summary>

```python 
def calculate_itl(wpm, words_per_token=0.75):
    ## NOTE: 60 seconds to a minute, 1000 milliseconds to a second

    ## NOTE: Unit arithmetic of [t/s] = [(words/min) * (tokens/word) * (min/s)] = [(words/min) / (words/tokens) / (s/min)]
    tokens_per_second = wpm / words_per_token / 60

    ## NOTE: Unit arithmetic of [ms/t] = [(ms/s * s/token)] = [(ms/s) / (token/s)]
    inter_token_latency_ms = 1000 / tokens_per_second

    return inter_token_latency_ms

print(f"READING:  230 words per minute correspond to {calculate_itl(230):.0f}ms between tokens, on average")  ## 196
print(f"SKIMMING: 700 words per minute correspond to {calculate_itl(700):.0f}ms between tokens, on average")  ## 64
```
</details>

<br>

**NOTES:** 
- In addition to defining a minimum threshold for comfort, you may also find it useful to identify a maximum threshold for speed in certain contexts. For example, a chat application with a lower-than-usual load might be so fast that it dumps output at an uncomfortable rate and causes the text view to scroll automatically, making it uncomfortable to read. For these cases, artificially sleeping between or iterating over streaming yields may be desirable. 
- [**LLMPerf**](https://github.com/ray-project/llmperf), another common benchmarking SW, incorporates first-token latency into inter-token latency computation. Beware of comparing directly the results from different benchmarking tools.

<hr><br>

## **Introducing the Simulator**

In this notebook, you will leverage and further develop a simulator that models how Tensor-LLM assembles the requests into batches. For now, let's use the default parameters and review the output.

In [2]:
import plotly.io as pio
pio.renderers.default = "iframe"

import simulator as sim
import numpy as np

engine = sim.Engine(
    max_batch_size = 2, # creates two slots in the batch
    load_generator = sim.BatchLoadGenerator(initial_batch=1), # it sends only one request at engine.current_time == 0
    batcher = sim.StaticBatcher()
)
engine.run(time_limit=10)
engine.plot_data.show()

You see two plots above. In the top plot, you can see batch composition depending on time. Each column is one LLM evaluation from the first layers to the last. The color of the cell and the letter represent the current stage of the request in the batch:
* **p**refill (pink)
* **d**ecoding (yellow) 
* **e**mpty slot (blue)

The width of the **p**refill cell is equal to the TTFT ($=2$) and the width of the **d**ecoding cell corresponds to ITL ($=1$). Make sure to hover your mouse over the cells to get detailed information about the execution status, including width in ticks.

In the bottom plot, you can see the measured latencies at the moments of measurement. For example, the pink $(2, 2)$ point represents the TTFT measurement of our request: at the time of $2$ ticks the prefill has been completed and it took $2$ ticks for our request. Similarly, we represent the E2E latency of $5$ measured at $5$ ticks by the green-yellow point. Additionally, we show the current queue size by the orange line. Since we send only one request in this example, it is always equal to 0. Note that the latency scale uses the left y-axis and the queue size uses the right one. The x-axis is shared between both plots.

<br><hr>

## **Batching: The Key to Efficient Throughput**

GPUs are very good at processing highly-parallelized and concurrent tasks. For example, an NVIDIA H100 GPU has 16,896 FP32 Cores per GPU and 528 Tensor Cores organized into 132 streaming multiprocessors (SMs) ([**see the datasheet**](https://nvdam.widen.net/s/95bdhpsgrs/nvidia_h100_tensor_core_gpu_architecture_whitepaper_v1.03#page=39)), where each core can execute an independent thread of mathematical computation and each SM can parallelize hundreds of threads (either core-enabled math operations or parallelizable memory operations) at a time. The most efficient way to utilize the GPU is to make sure all the SMs always have something to compute and some memory operations to run at any given time.  

### Memory-Bound and Compute-Bound Functions

Now consider a simplified model where a function reads its input from memory, performs math operations, and then writes its output to memory. Let's assume $T_{mem}$ time is spent in accessing memory and $T_{math}$ time is spent performing math operations. If we further assume that memory and math portions of different threads can be overlapped: 
- The total time for the function is $max(T_{mem}, T_{math})$, and the longer of the two times demonstrates what limits performance.
- If math time $T_{math}$ is longer we say that a function is `math limited` or `compute-bound`. 
- If memory time $T_{mem}$ is longer then it is `memory limited` or `memory-bound`. 

For more low-level details see [**NVIDIA Deep Learning Performance Documentation**](https://docs.nvidia.com/deeplearning/performance/index.html) and in particular [**GPU Performance Background User's Guide**](https://docs.nvidia.com/deeplearning/performance/dl-performance-gpu-background/index.html), but for now let's connect these topics to the LLM operations we take for granted in typical use.

<br>

### **Prefill:** A Compute-Bound Operation

**During prefill, most operations are compute-bound.**
- Propagating the initial context requires larger matrices to interact to resolve attention across the entire prefill context.
- The intermediate results of the calculation are written to KV-cache (cached attention matrix values stored in memory), but that requires few memory operations.
- Compute-bound property generally manifests after a certain prefill token limit (in our tests, around 300 tokens or more cause $T_{math} > T_{mem}$ on our GPU setup).

<br>

### **Decoding:** A Memory-Bound Operation

**During decoding, most operations are memory-bound.**
- Generating one token at a time means the input for each next token is a single embedded token and many cached components, which results in small matrix operations during forward propagation. 
- The KV-cache from previously provided/generated tokens continues to grow, so retrieving requires more resources. 

To improve the efficiency, you need to increase computations per read byte of memory. The simplest way to do it is by batching the requests together. With the batch size `b` the system can load the weights of the LLM from the GPU memory to the SMs once, but compute `b` next tokens.

<br>


### Arithmetic Intensity

To help support this model, **arithmetic intensity** is a common metric for evaluating the compute-boundedness of a given function. It is defined as the ratio of floating-point operations to the number of data elements accessed by the function - usually in FLOPs/byte - with a high arithmetic intensity indicating a high computational load. 

Below is a figure from the paper [**SARATHI: Efficient LLM Inference by Piggybacking Decodes with Chunked Prefills**](https://arxiv.org/pdf/2308.16369) demonstrating the arithmetic intensity of prefills and decodes for LLaMA-13B on A6000 GPU. The different colors represent different operations within the transformer block: *preproj* for preprojection, a single matrix multiplication; *attn* for attention computation, *preproj* for postprojection, and *ffn* for feed-forward network.

<img src="images/sarathi.png" alt="Arithmetic Intensity" width=1200px/>
<br>

### Simulating Infinitely Compute-Bound Prefill and Infinitely Memory-Bound Decoding

Using the simulator, we can model our properties of interest to their extremes and see how our system performs in asymptotic cases. 
* Batching together $N$ prefills in one slot takes $N\times$ the time.
* Batching together as many decodings won't affect the ITL. 

Optional: Feel free to uncomment the next line and review the code of duration estimation. Note that we sum up the $T_{math}$ from prefills and compute max between $T_{mem}$ from decodings.

In [3]:
sim.Engine.get_current_batch_duration??

Signature: sim.Engine.get_current_batch_duration(self) -> float
Docstring: <no docstring>
Source:   
    def get_current_batch_duration(self) -> float:
        decoding_requests = self.get_decoding_requests()
        # for no chunking the line below is equivalent to
        # prefill_time = sum([req.prefill_time for req in self.get_prefilling_requests()])
        prefill_time = sum([req.get_current_duration() for req in self.get_prefilling_requests()])
        itl_time = max([req.itl for req in decoding_requests]) if decoding_requests else 0
        return max(prefill_time, itl_time, 1.) # 1. is the minimal step duration
File:      /dli/task/simulator/engine.py
Type:      function



Let's now simulate 2 concurrent requests, submitted at time 0.

In [4]:
engine = sim.Engine(
    max_batch_size = 2, # creates two slots in the batch
    load_generator = sim.BatchLoadGenerator(initial_batch=2), # it sends 2 requests at engine.current_time == 0
    batcher = sim.StaticBatcher()
)
engine.run(time_limit=10)
engine.plot_data.show()

As expected, now both requests have TTFT of $2*2=4$ but the ITL is still $1$.

### Adding Variable Output Length
Now let's increase our output length to $10$ tokens and set its standard deviation to $5$ to simulate variability in the expected response lengths. We also increase simulation `time_limit` to $100$ and `max_batch_size` to $4$:

In [5]:
np.random.seed(42)
load_generator = sim.BatchLoadGenerator( 
    initial_batch=100, # how many requests it sends at engine.current_time == 0
    target_output_len_tokens=10,
)
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.StaticBatcher()
)
engine.run(time_limit=100)
engine.plot_data.show()

When running this, you can see that there are a lot of **e**mpty slots which indicate that the GPUs aren't utilized efficiently. But how do we measure this?

<br><hr>

## **Throughput Metrics**

One of the advantages of using latency metrics is their ease of interpretation and lack of ambiguity. They can be measured regardless of the inference system's organization, and they require no awkward normalization to become immediately interpretable! 

However, to get from latency benchmarks to GPU count estimations, one also needs some throughput metrics. These metrics measure the capacity of a system to process data per unit of time. Here's a breakdown of the various throughput metrics and their implications:

- **Tokens per second per model instance:**
    - **Across all phases:** Measures the total processing capability of a single model instance, including pre-processing, generation, and post-processing stages.
    - **Only in the generation phase (a.k.a 1/ITL):** Focuses on the model's ability to generate tokens, offering a direct measure of the model’s generative performance.

- **Tokens per second per GPU:**
    - This metric can be specified for either only the generation phase or all phases, indicating how effectively a GPU is being utilized to process tokens. This helps in assessing the efficiency of the GPU in handling specific tasks within the inference pipeline.

- **Tokens per second per server:**
    - Similar to the per GPU metric but scaled up to the server level. This measures the overall throughput of an entire server, which may contain multiple GPUs and model instances. It's crucial for evaluating server-level performance and infrastructure scalability.

- **Prompts per second:**
    - **Per Model Instance:** This measures how many complete prompts a single model instance can handle in one second, providing a straightforward metric of model instance efficiency.
    - **Per GPU:** Reflects the number of prompts a GPU can process per second, useful for gauging GPU performance in a real-world application scenario.
    - **Per Server:** Measures the capacity of a server to handle prompts, indicating the throughput at the server scale.

- **Concurrent Metrics:**
    - **Concurrent Requests:** Refers to the number of requests a system can handle at the same time. It's a critical measure of system robustness and concurrency handling.
    - **Concurrent Clients:** Indicates how many clients can simultaneously interact with the system without degrading performance, essential for understanding the scalability of client-server interactions.

### Choosing The Right Throughput Metric

Often, benchmarking software shortens the units to just `tokens/second`, leading to ambiguity about the applied normalization. For the purposes of sizing, the most convenient throughput metric is `prompts/second/server`, which allows benchmarkers to choose between different combinations of tensor parallelism strategies (from now on `TP`) with the number of servers being a natural parameter. In our established benchmarks, we normalize by the standard servers with 8 GPUs, meaning that we consider the throughput of 2 instances of TP4 or 8 instances of TP1. This metric also highlights the dependence of the throughput on the specific composition of requests, including input and output lengths.

### **[EXERCISE]** Measuring Throughput and Printing Metrics

In our simulation, we have an `engine.plot_data.metrics` object implemented in [./simulator/metrics.py](./simulator/metrics.py). Every time a new request completes, we record a pair of `(current time, E2E Latency)` to `metrics.e2e_latency`. The `metrics.get_e2e_latencies()` returns just the measured values. Complete the `print_experiment_metrics` function below to quantify the speedups. Please, don't change any formatting for now, so that the helper function can verify your implementation is correct, see the cell below.

In [6]:
import numpy as np

def print_experiment_metrics(engine):
    print("# Experiment Config:")
    print(f"load_generator = {str(engine.load_generator)}")
    print(f"batcher = {str(engine.batcher)}")
    metrics: sim.Metrics = engine.plot_data.metrics
    # we record the latency of every completed request
    e2e_latencies = metrics.get_e2e_latencies()
    ttfts = metrics.get_ttfts()
    itls = metrics.get_itls()

    print("\n# Latency Metrics:")
    print(f"Average E2E Latency: {np.mean(e2e_latencies):.2f}")
    print(f"Average TTFT: {np.mean(ttfts):.2f}")
    print(f"Average ITL: {np.mean(itls):.2f}")
    print(f"Median E2E Latency: {np.percentile(e2e_latencies, 0.5):.2f}")
    print(f"Median TTFT: {np.percentile(ttfts, 0.5):.2f}")
    print(f"Median ITL: {np.percentile(itls, 0.5):.2f}")

    print("\n# Throughput Metrics:")
    num_requests: int = len(e2e_latencies)
    run_time: float = metrics.times[-1]

    # TODO: calculate the throughput in requests/(1000 ticks)/instance
    # You have num_requests completed in run_time
    # How many complete in 1 tick on average
    # How many complete in 1K ticks on average
    requests_per_1k_ticks_per_instance: float = -1
    
    print(f"Requests/(1K ticks)/instance = {requests_per_1k_ticks_per_instance:.2f}")

    current_batch_tokens = sum(req.tokens_generated for req in engine.current_batch.values())
    total_tokens_generated = sum(metrics.get_osls()) + current_batch_tokens
    tokens_per_1k_ticks_per_instance = 1000 * total_tokens_generated / run_time
    print(f"Tokens/(1K ticks)/instance = {tokens_per_1k_ticks_per_instance:.2f}")

print_experiment_metrics(engine)

# Experiment Config:
load_generator = BatchLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=1
     initial_batch=100
     target_output_len_std=5
)
batcher = StaticBatcher

# Latency Metrics:
Average E2E Latency: 58.16
Average TTFT: 52.80
Average ITL: 1.00
Median E2E Latency: 16.27
Median TTFT: 8.00
Median ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = -1.00
Tokens/(1K ticks)/instance = 1680.00


<details>
<summary><b>Expected Results:</b></summary>

```python
# Experiment Config:
load_generator = BatchLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=1
     initial_batch=100
     target_output_len_std=5
)
batcher = StaticBatcher

# Latency Metrics:
Average E2E Latency: 58.16
Average TTFT: 52.80
Average ITL: 1.00
Median E2E Latency: 16.27
Median TTFT: 8.00
Median ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = 190.00
Tokens/(1K ticks)/instance = 1680.00
```
</details>

<br>

Note that due to the structure of the load generator, the latency metrics do not make much sense here. You will later learn how to simulate more realistic loads where requests come independently in accordance with some underlying distribution.

Since the metrics are very important, we've implemented a function to check your implementation and to print diff between your output, and the expected output. See below.

<details>
<summary><b>Reveal Solution</b></summary>

```python 
import numpy as np

def print_experiment_metrics(engine):
    print("# Experiment Config:")
    print(f"load_generator = {str(engine.load_generator)}")
    print(f"batcher = {str(engine.batcher)}")
    metrics: sim.Metrics = engine.plot_data.metrics
    # we record the latency of every completed request
    e2e_latencies = metrics.get_e2e_latencies()
    ttfts = metrics.get_ttfts()
    itls = metrics.get_itls()

    print("\n# Latency Metrics:")
    # TODO: calculate np.mean of the latencies list
    print(f"Average E2E Latency: {np.mean(e2e_latencies):.2f}")
    print(f"Average TTFT: {np.mean(ttfts):.2f}")
    print(f"Average ITL: {np.mean(itls):.2f}")

    # Otional TODO: calculate median of latencies using np.percentile(array, percentile_value)
    print(f"Median E2E Latency: {np.percentile(e2e_latencies, 0.5):.2f}")
    print(f"Median TTFT: {np.percentile(ttfts, 0.5):.2f}")
    print(f"Median ITL: {np.percentile(itls, 0.5):.2f}")

    print("\n# Throughput Metrics:")
    num_requests: int = len(e2e_latencies)
    run_time: float = metrics.times[-1]

    # TODO: calculate the throughput in requests/(1000 ticks)/instance
    # You have num_requests completed in run_time.
    # How many complete in 1 tick on average
    # How many complete in 1K ticks on average
    requests_per_1k_ticks_per_instance: float = -1
    requests_per_1k_ticks_per_instance: float = 1000.*num_requests/run_time

    print(f"Requests/(1K ticks)/instance = {requests_per_1k_ticks_per_instance:.2f}")

    current_batch_tokens = sum(req.tokens_generated for req in engine.current_batch.values())
    total_tokens_generated = sum(metrics.get_osls()) + current_batch_tokens
    tokens_per_1k_ticks_per_instance = 1000 * total_tokens_generated / run_time
    print(f"Tokens/(1K ticks)/instance = {tokens_per_1k_ticks_per_instance:.2f}")

```
</details>

In [7]:
from simulator.extra import check_print_metrics

# set show median to True if you have implemented the optional TODO
check_print_metrics(print_experiment_metrics, engine, show_median=False) 

--- Your Implementation

+++ Reference

@@ -13,11 +13,8 @@

 Average E2E Latency: 58.16
 Average TTFT: 52.80
 Average ITL: 1.00
-Median E2E Latency: 16.27
-Median TTFT: 8.00
-Median ITL: 1.00
 
 # Throughput Metrics:
-Requests/(1K ticks)/instance = -1.00
+Requests/(1K ticks)/instance = 190.00
 Tokens/(1K ticks)/instance = 1680.00
 


<hr><br>

## **Inflight Batching (IFB)**

IFB is a technique used during LLM inference to balance GPU memory with compute utilization and reduce latency.

During auto-regressive inference, the LLM is evaluated from the first layers to the last for every token to generate, using previous tokens to generate the next ones. The process involves:

* The first call to the LLM producing the prefill token
* Subsequent calls generating the decoding tokens

IFB enables sequences at different stages (prefill and decoding) to be processed within the same batch, without requiring all requests to be completed before the next one can enter the batch.

**Key Benefits of IFB:**

* Allows for a nearly constant batch size for each token, resulting in higher GPU utilization
* Enables new request execution to start quicker when slots are available, as the scheduler only needs to wait for the generation of the next token, not the completion of current requests

See the illustration below for a visual representation of in-flight batching in TensorRT-LLM:

<img src="images/ifb_trt-llm.png" alt="In-flight batching in TensorRT-LLM" width=1200px/>

<br>

### IFB In Simulation
Let's enable IFB in our simulation. First, let's review how `StaticBatcher` works, which cannot batch together prefills and decodings.

In [8]:
sim.StaticBatcher.add_requests??

Signature: sim.StaticBatcher.add_requests(self)
Docstring: <no docstring>
Source:   
    def add_requests(self):
        engine = self.engine
        if engine.get_occupied_slots(): return # static batcher cannot batch together new prefills with old decodings
        for slot in engine.get_all_slots():
            if not len(engine.queue): # checking if we still have more requests to run
                break
            request = engine.queue.pop(0)
            engine.assign_request_to_slot(request, slot)
File:      /dli/task/simulator/batcher.py
Type:      function

As soon as all slots are free, it tries to fill all of them. Now, let's see what changes in the `IFBatcher`:

In [9]:
sim.IFBatcher.add_requests??

Signature: sim.IFBatcher.add_requests(self)
Docstring: <no docstring>
Source:   
    def add_requests(self):
        engine = self.engine
        empty_slots = engine.get_all_slots() - engine.get_occupied_slots()
        for slot in empty_slots:
            if not len(engine.queue):
                break
            request = engine.queue.pop(0)
            engine.assign_request_to_slot(request, slot)
File:      /dli/task/simulator/batcher.py
Type:      function

It fills the slots as soon as they're empty. Let's try it in our previous scenario.

In [10]:
np.random.seed(42)
load_generator = sim.BatchLoadGenerator( 
        initial_batch=100, # how many requests it sends at engine.current_time == 0
        target_output_len_tokens=10,
    )
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.IFBatcher()
)
engine.run(time_limit=100)
engine.plot_data.show()
print_experiment_metrics(engine)

# Experiment Config:
load_generator = BatchLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=1
     initial_batch=100
     target_output_len_std=5
)
batcher = IFBatcher

# Latency Metrics:
Average E2E Latency: 58.44
Average TTFT: 52.90
Average ITL: 1.39
Median E2E Latency: 16.52
Median TTFT: 8.00
Median ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = -1.00
Tokens/(1K ticks)/instance = 2376.24


The `Requests/(1K ticks)/instance` are now $267.33$ instead of $190.00$. The effect is more dramatic the bigger the `max_batch_size`.


### Chunked Context
To optimize performance, you can separate the prefill into chunks and batch together one chunk of prefill and multiple decodings to attempt a balance between $T_{mem}$ and $T_{math}$. This technique is implemented in TensorRT-LLM as [**Chunked Context**](https://nvidia.github.io/TensorRT-LLM/advanced/gpt-attention.html#chunked-context). It is important to keep chunks large enough to still be able to reach compute-boundness.

For our simulation, we assume that our prefills are long enough such that splitting each one into $2$ chunks would still keep us compute-bound. For now, let's keep our batcher as-is to allow for more than $1$ prefill chunk per batch.

In [11]:
np.random.seed(42)
load_generator = sim.BatchLoadGenerator( 
        initial_batch=100, # how many requests it sends at engine.current_time == 0
        target_output_len_tokens=10,
        total_prefill_chunks=2, # into how many chuncks can our prefills be split
    )
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.IFBatcher()
)
engine.run(time_limit=100)
engine.plot_data.show()
print_experiment_metrics(engine)

# Experiment Config:
load_generator = BatchLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     initial_batch=100
     target_output_len_std=5
)
batcher = IFBatcher

# Latency Metrics:
Average E2E Latency: 57.42
Average TTFT: 54.51
Average ITL: 1.14
Median E2E Latency: 16.45
Median TTFT: 8.00
Median ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = -1.00
Tokens/(1K ticks)/instance = 2730.00


The `Requests/(1K ticks)/instance` results are now $310.00$ instead of $267.33$, which means we were able to decode more tokens during the same prefill phases using the chunked context strategy!

### **[EXCERCISE]** Batcher With Only One Prefill Per Batch

Given the implementations you saw earlier, complete the `IFBatcherWithOnePrefillOnly` class so that every chunk can contain only one prefill request at a time (i.e. `req.is_in_prefill() == True`):

In [12]:
from typing import List

class IFBatcherWithOnePrefillOnly(sim.IFBatcher):

    def add_requests(self):

        engine = self.engine
        prefilling_requests: List[sim.Request] = engine.get_prefilling_requests()

        ## TODO: check if any requests are already prefilling. If yes, return. 
        ## See engine.py for engine querying methods.

        empty_slots = engine.get_all_slots() - engine.get_occupied_slots()
        for slot in empty_slots:
            if not len(engine.queue):
                break
            req = engine.queue.pop(0)
            engine.assign_request_to_slot(req, slot)

            ## TODO: Make sure no more than one request is added. 
            ## Otherwise next batch will contain two prefills.

## Ground truth implementation accessible below:
# IFBatcherWithOnePrefillOnly = sim.IFBatcherWithOnePrefillOnly

In [13]:
np.random.seed(42)

load_generator = sim.BatchLoadGenerator( 
    initial_batch=100, # how many requests it sends at engine.current_time == 0
    target_output_len_tokens=10,
    total_prefill_chunks=2, # this is where we set, into how many chuncks can our prefills be split
)

load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = IFBatcherWithOnePrefillOnly()
)

engine.run(time_limit=100)
engine.plot_data.show()
print_experiment_metrics(engine)

# Experiment Config:
load_generator = BatchLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     initial_batch=100
     target_output_len_std=5
)
batcher = IFBatcherWithOnePrefillOnly

# Latency Metrics:
Average E2E Latency: 57.42
Average TTFT: 54.51
Average ITL: 1.14
Median E2E Latency: 16.45
Median TTFT: 8.00
Median ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = -1.00
Tokens/(1K ticks)/instance = 2730.00


If you implemented the class correctly, the `Requests/(1K ticks)/instance` results are now $360.00$ instead of $310.00$! This is the power of balancing $T_{mem}$ and $T_{math}$

### Client-Side Concurrency

For the IFB engine, the batch size can change after every token, whenever a new request comes or a current one completes. Irregular batch sizes introduce variability when measuring latencies. To achieve more stable latencies, we use client-side concurrency.

#### How It Works
* If concurrency is set to `C`, the client sends `C` requests simultaneously. 
* As soon as it gets any request final response,  the client sends another request to maintain the concurrency level.
* At any given time, the client has exactly `C` outgoing concurrent requests.

Client-side concurrency has been implemented in various tools, including [**Triton Performance Analyzer**](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/client/src/c%2B%2B/perf_analyzer/docs/inference_load_modes.html#concurrency-mode) and in 
[**GenAI-Perf**](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/perf_analyzer/genai-perf/README.html#concurrency-int). 

#### How We Simulate It

We simulate client-side concurrency using  `sim.ConcurrentLoadGenerator`. Reviewing the source code, we can get a good sense of the load generation strategy:

In [14]:
sim.ConcurrentLoadGenerator.generate_load??

Signature: sim.ConcurrentLoadGenerator.generate_load(self)
Docstring: <no docstring>
Source:   
    def generate_load(self):
        current_concurrency = len(self.engine.get_occupied_slots())
        already_in_queue = len(self.engine.queue)
        ## We want to reach target concurrency but not overshoot it, so limit queue buffer
        need_to_add = self.target_concurrency - current_concurrency - already_in_queue
        self.add_n_requests_to_queue(need_to_add)
        self.last_generation_time = self.engine.current_time
File:      /dli/task/simulator/load_generator.py
Type:      function

Let's replace our load generator with this one and select `target_concurrency=3`. Note that this is less than `max_batch_size`, which should make a noticeable effect.

In [15]:
np.random.seed(42)
load_generator = sim.ConcurrentLoadGenerator( # we've updated the generator class
    target_concurrency=3, 
    target_output_len_tokens=10,
    total_prefill_chunks=2, # how many chuncks can our prefills be split into
)

load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = IFBatcherWithOnePrefillOnly()
)

engine.run(time_limit=100)
engine.plot_data.show()
sim.extra.print_experiment_metrics(engine)

# Experiment Config:
load_generator = ConcurrentLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     target_concurrency=3
     target_output_len_std=5
)
batcher = IFBatcherWithOnePrefillOnly

# Latency Metrics:
Average E2E Latency: 11.00
Average TTFT: 3.19
Average ITL: 1.06

# Throughput Metrics:
Requests/(1K ticks)/instance = 260.00
Tokens/(1K ticks)/instance = 2240.00


You can see that this is the first example where the queue size and the E2E latency are stationary during the measurements: they do not grow or decline constantly, as you saw in the previous cases. This finally allows us to make sense of the latencies.

### Max Batch Size

TensorRT-LLM engines have two parameters called `max_batch_size`:

- One is set for the engine build and is used during the kernel selection process to make sure the resulting batch-size-capable system fits into memory.
- One is set for runtime and specifies how many requests can be batched together. This is the one we use in our simulation.

Note that the second one should be less than or equal to the first one. See the [docs](https://nvidia.github.io/TensorRT-LLM/performance/perf-best-practices.html#max-batch-size) for details.

#### Batch Size and Concurrency

Let's assume we've set a runtime `max_batch_size` to value $MBS$ and we're running our benchmark with concurrency $C$. If $C < MBS$ as we set before, the engine has free slots available in the batches and is usually running with batch size $C$. If $C >= MBS$, then the engine is usually running with no free slots and the batch size for most of the batches is $MBS$. The queue size is then $C - MBS$ on average.

Note that the real client doesn’t have to care about available slots in the server: the batch is almost static anyway. For details on how this happens, see appendix [below](#On-concurrency-and-request-rate-as-an-input-parameter).

Let's have a look at the example, where $ C = 6 > MBS = 4$
<br>

In [16]:
np.random.seed(42)
load_generator = sim.ConcurrentLoadGenerator( 
        target_concurrency=6, # increased from last time
        target_output_len_tokens=10,
        total_prefill_chunks=2, # how many chuncks can our prefills be split into
    )
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.IFBatcherWithOnePrefillOnly()
)
engine.run(time_limit=100)
engine.plot_data.show()
sim.extra.print_experiment_metrics(engine)

# Experiment Config:
load_generator = ConcurrentLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     target_concurrency=6
     target_output_len_std=5
)
batcher = IFBatcherWithOnePrefillOnly

# Latency Metrics:
Average E2E Latency: 15.14
Average TTFT: 7.87
Average ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = 360.00
Tokens/(1K ticks)/instance = 3170.00


You may recall we got the same maximum throughput of $360$ from our static batching exercise, but we can now measure the TTFT and E2E latency at which such throughput can be achieved.




<br><hr>

## **Additional Notes**

---

### On concurrency and request rate as a result metric

To effectively measure system performance, it's essential to consider throughput, end-to-end latency, and concurrency. A hypothetical server capable of handling 60 simultaneous requests with an e2e latency of 20 seconds each, achieves a throughput of 3 requests per second. This throughput reflects the system’s ability to process multiple requests concurrently, offsetting the high latency of individual requests.

Comparatively, consider two systems: one with a batch size of 60 and a 20-second latency, and another with a batch size of 30 and a 10-second latency. Both process 3 requests per second on average, but the latter provides faster responses, demonstrating superior efficiency despite its lower concurrency.

Thus, we recommend using requests per minute as the primary metric for system sizing and communication with stakeholders. This metric ensures a clear and balanced understanding of system capacity and creates an opportunity to factor in concurrency/latency requirements.

```python
requests_per_second = concurrent_users * requests_per_session / session_duration_in_seconds
requests_per_minute = 60 * requests_per_second
```

---

### On concurrency and request rate as an input parameter

For good speed measurements, we need engine batch size to be constant from token to token. Below we explain why using concurrency as an input for speed measurements helps us achieve it, and why one should avoid using the request rate as an input parameter.

Let's simulate some moderate request rate of one request in 5 ticks.

In [17]:
np.random.seed(42)
load_generator = sim.RequestRateLoadGenerator( 
        request_rate=1./5., # 1 request in 5 ticks
        target_output_len_tokens=10,
        total_prefill_chunks=2, # this is where we set, into how many chuncks can our prefills be split
    )
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.IFBatcherWithOnePrefillOnly()
)
engine.run(time_limit=100)
engine.plot_data.show()
sim.extra.print_experiment_metrics(engine)

# Experiment Config:
load_generator = RequestRateLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     request_rate=0.2
     target_output_len_std=5
)
batcher = IFBatcherWithOnePrefillOnly

# Latency Metrics:
Average E2E Latency: 10.06
Average TTFT: 2.00
Average ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = 170.00
Tokens/(1K ticks)/instance = 1650.00


In this case, the queue is 0 all the time, and our latency measurements can be similar to reality. Note two things about these measurements:
1. A much longer warmup period is observed when the engine is running with a low batch size.
2. The resulting throughput difference, where $1000 * 1 / 5 = 200$ requests/1k ticks is expected but $170$ is observed.

Now, let's consider what happens when our request rate exceeds the capability of our engine by trying a rate of $460$:

In [18]:
np.random.seed(42)
load_generator = sim.RequestRateLoadGenerator( 
        request_rate=460./1000., # 460 requests in 1000 ticks
        target_output_len_tokens=10,
        total_prefill_chunks=2, # this is where we set, into how many chuncks can our prefills be split
    )
load_generator.target_output_len_std = 5

engine = sim.Engine(
    max_batch_size = 4, # how many slots in the batch
    load_generator=load_generator,
    batcher = sim.IFBatcherWithOnePrefillOnly()
)
engine.run(time_limit=100)
engine.plot_data.show()
print(f"Final Queue: {len(engine.queue)}")
sim.extra.print_experiment_metrics(engine)

Final Queue: 6
# Experiment Config:
load_generator = RequestRateLoadGenerator(
     prefill_time=2
     itl=1
     target_output_len_tokens=10
     total_prefill_chunks=2
     request_rate=0.46
     target_output_len_std=5
)
batcher = IFBatcherWithOnePrefillOnly

# Latency Metrics:
Average E2E Latency: 17.66
Average TTFT: 11.03
Average ITL: 1.00

# Throughput Metrics:
Requests/(1K ticks)/instance = 350.00
Tokens/(1K ticks)/instance = 3060.00


This is the key caveat of using the request rate. In this situation, there are no free slots available. While the throughput is pushed to the system's limit, the client keeps sending more and more requests and the queue grows with the approximate speed of $460 - 360 = 100$ requests per 1k ticks. The final average TTFT depends on the duration of the measurement and final queue size, but it is impossible to identify the actual latency of the requests at that point. That's why we recommend avoiding request rate as a measurement input. In a real-world scenario, the system tries to autoscale if it reaches high enough latencies and has the resources to do so.

**Overall we saw 5 use cases represented:**

1. **Static batching with MBS:** The prefills are batched together and don't benefit from the overlap of computations and memory-intensive operations. The E2E and TTFT latency is high. In reality, in static batching the requests wait on average E2E Latency$/ 2$ in the queue. This adds to both real TTFT and E2E Latency. That's why nowadays static batching is almost universally deprecated for LLM inference.
2. **Concurrency < MBS:** There are almost no free slots available and no queue is forming. The engine is running with a maximum throughput of 280 requests/1000 ticks. TTFT is 2.67 and E2E Latency is 10.14.
3. **Concurrency > MBS:** There are almost no free slots available and a queue of size 2 is forming. The engine is running with a maximum throughput of $360$ requests/100 ticks. However, TTFT is growing with the queue size. It is now 7.87 because every request has to wait until the slots are available. E2E Latency grows with the TTFT and is now 15.14
4. **Request Rate < Maximum Throughput:** In this situation there are free slots available. The situation is almost stationary and batch size is almost constant and equal to 3. The throughput is closely defined by the request rate we set. This is a valid measurement.
5. **Request Rate > Maximum Throughput:** Correct maximum throughput, but invalid measurements. See the details above.

### **[OPTIONAL EXCERCISE]** Queue Growth

Create longer simulations (10000 ticks) for the same setups with `RequestRateLoadGenerator` and with `ConcurrentLoadGenerator`. 
* Avoid plotting them, to keep the memory consumption reasonable (comment out `engine.plot_data.show()`)
* Check the queue size at the end of the execution using `len(engine.queue)`
* Compare TTFT with 100 ticks as the time limit as above with 10000 ticks in both cases
* Compare E2E Latency with 100 ticks as the time limit as above with 10000 ticks in both cases

In [19]:
# TODO Create batcher
# TODO Create ConcurrentLoadGenerator
# TODO Create engine
# TODO Run engine to 100 ticks and print metrics
# TODO Run engine to 10000 ticks and print metrics
# TODO Create ConcurrentLoadGenerator
# TODO Create another engine
# TODO Run engine to 100 ticks and print metrics
# TODO Run engine to 10000 ticks and print metrics

<br>

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>